In [11]:

from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed, IntSlider
from skimage.io import imread

from skimage.measure import regionprops
from skimage.color import label2rgb
from shapely.geometry import polygon, box


In [12]:
def metrics(truth,prediction):
    """
    Perform metrics for semantic segmentation: IoU, PCC, recall, precision, DSC and Cohen's kappa.
    :param truth: a ground truth image
    :param prediction: a prediction of the semantic segmentatio
    :return: 6 different values for the metrics
    """

    truth = np.float32(truth)
    
    if prediction.dtype == np.uint8:
        prediction = prediction /255.


    FP = np.sum(np.logical_and(np.logical_not(truth),prediction))
    FN = np.sum(np.logical_and(truth,np.logical_not(prediction)))
    TP = np.sum(np.logical_and(truth,prediction))
    TN = np.sum(np.logical_and(np.logical_not(truth),np.logical_not(prediction)))

    FP = FP.astype(np.uint64)
    FN = FN.astype(np.uint64)
    TP = TP.astype(np.uint64)
    TN = TN.astype(np.uint64)

    
    pearson = np.corrcoef(truth.ravel(), prediction.ravel())[0,1]
    
    recall = TP/(TP+FN)
    precision = TP/(TP+FP)
    
    IOU = TP/(TP+FP+FN)
    
    DSC = 2* TP/(2*TP + FP + FN)
    
    kappa = 2*(TP*TN-FN*FP)/((TP+FP)*(FP+TN)+(TP+FN)*(FN+TN))
    
#     PCC = (TP*TN-FP*FN)/(np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))) # works, but might give an overflow
    return IOU, pearson,recall,precision, DSC, kappa

In [13]:
# loop through all images and print the semantic segmentaion metrics

for i in tqdm(range(40)):
    truth = imread('../wetransfer-3c006d/_REPOSITORY/png/masks_resized1024/'+ str(i)+ '.png')
    prediction = imread('../Random Mess/Final/compare/SLIC/'+ str(i)+ '.png')
    
    prediction = prediction > 0

    prediction = prediction[:,:,0]*1.
    
    print(metrics(truth,prediction))




  0%|          | 0/40 [00:00<?, ?it/s]

(0.9658135981665393, 0.9825258310890788, 0.9806088811324414, 0.9846183800623053, 0.9826095404643933, 0.9825237659816722)
(0.8904109589041096, 0.9417576532419328, 0.9372564302416212, 0.9468503937007874, 0.9420289855072463, 0.9417453239860973)
(0.9172614858751382, 0.9569544238895592, 0.9995617879053462, 0.917630493814744, 0.9568454721828955, 0.9560475458228062)
(0.8251940896568996, 0.9042398421302769, 0.8578495183545951, 0.9559036843632144, 0.9042261251372119, 0.9028800297342878)
(0.7603009673951988, 0.8656078020270536, 0.9331574318381706, 0.804092459264873, 0.8638306533686139, 0.8631931964186513)
(0.9497222222222222, 0.9742533883823069, 0.9901534897190849, 0.9587773415591699, 0.9742128508334521, 0.9741262759932905)
(0.9031715161941806, 0.9487271409099444, 0.9904437699942581, 0.9111111111111111, 0.9491225656847525, 0.9478595905300864)
(0.8185247661401626, 0.8984003515396224, 0.8591018831482375, 0.9454442397619385, 0.9002074443863525, 0.8973107470568394)
(0.9043722927696124, 0.94887978497

In [15]:
def get_centers(mask):
    """
    Returns the centers of sections from an annotated grayscale mask, in which each section is given a separate value.
    :param mask: a grayscale mask with sections with different values
    :return: a numpy array with the centers of all sections
    """
    values = np.unique(mask)
    centers = []
    for i in values:
        sections = mask.copy()
        sections[sections != i] = 0

        labeled_foreground = (sections > 0).astype(int)
        properties = regionprops(labeled_foreground, sections)

        try:
            center_of_mass = properties[0].centroid
            center_of_mass = np.array(center_of_mass)
            centers.append(center_of_mass)

        except:
            pass
    return centers

In [20]:
def find_nearest_vector(array, coordinates):
  """
    Return the nearest center, given a list of possible centers to choose from and coordinates
  :param array: A numpy array of possible centers to choose from
  :param coordinates: The coordinates of the found center
  :return: The actual center closest to the center found
  """

  idx = np.array([np.linalg.norm(x+y) for (x,y) in array-coordinates]).argmin()
  return array[idx]


def compare(prediction_filepath, imagen0):
    """
Compare the location of sections between a prediction and the truth
    :param prediction_filepath: the filepath to the prediction images
    :param imagen0: the image number
    :return: the number of correct sections (TP), the number of incorrect sections (FP) and the number of missed sections (FN)
    """

    # the centers of the 40 input images, for now hardcoded in. Note these we taken at 2944x2944, so resizing is needed
    centers = [[[1505.229962271096, 1676.302659427625], [1440.6879240162823, 1866.2347354138399], [1036.7799122807019, 1877.4659649122807], [939.163248564397, 1724.8947224500957]], [[1720.2566936208445, 2065.628481581312], [1343.6255549635944, 1934.6664002841414], [1205.4495209368347, 1746.4292051100072], [1277.868247821878, 1581.6484995159728]], [[1559.4807012027356, 1935.954720540838], [1586.329206612818, 1810.1323318487205], [1617.418208676722, 1682.3403825221583], [1645.3335122287817, 1556.7376370466918], [1383.1130198212204, 2197.0478818499805], [1408.3323430834857, 2070.750837903717], [1438.3145308651783, 1939.250734885199], [1464.1014361300076, 1812.4085411942556], [1492.7641252817284, 1683.302945519546], [1522.6869123445556, 1554.852820746133], [1554.56693957115, 1424.1065107212476], [1584.2082460341214, 1297.2372792577073]], [[2329.575521049572, 1121.7554596030889], [2211.6688815060907, 1134.782690178039], [1995.404048784535, 1134.2509748610305], [1879.0149913344887, 1112.9459272097054], [1756.9211463596455, 1093.0709020127558], [1637.140499918314, 1082.641071720307], [1426.1082269033006, 1023.7832973959197], [1306.8471261422837, 1025.2518575454778], [1182.3584874774922, 1028.2294156163039], [1060.3672862453532, 1033.4209830648492], [939.9763461207638, 1034.7517632891795]], [[1123.7293640054127, 2049.666102841678], [1307.9847083926031, 2039.9281650071123], [1375.7296718972896, 1958.7114835948644], [1452.6158169409875, 1879.006021678041], [1484.1691204228011, 1725.370705926765], [1802.5844534910516, 1208.1761271987095], [1804.7406556857777, 1084.4846373139055]], [[1413.9482488238368, 1655.5250392054365], [1482.0493468795355, 1553.4391872278666], [1391.898643751941, 1527.5139248369396]], [[2257.5798365122614, 1922.684196185286], [2258.603144654088, 1839.053459119497], [2253.0674288380774, 1745.9121558562763], [2245.4233392800925, 1638.6997586315458], [2260.5465092189766, 1524.7336855189558], [1743.324749100549, 2015.3871425866314], [1696.903865955256, 1848.4840400636526], [1738.5883012521024, 1734.5512988226499], [1733.3080373831776, 1614.421121495327], [1683.4896136854968, 1483.5873672337625], [1690.8548881255222, 1367.9796676260328], [1622.187403250774, 1124.2315208978328], [1634.3111213235295, 1238.4046875], [1186.1025945550887, 2139.1018636945005], [1167.3352467364132, 2021.603462642348], [1182.3921276000374, 1849.85505083481], [1184.1258247374778, 1719.0676517052318], [1067.8902188142883, 1483.8182158219563], [1093.8294243070363, 1598.5943265041253], [1017.6191133186167, 1269.7127483443708], [1108.7791286727456, 1346.8386294556508]], [[2241.854032613486, 1372.3827236668135], [2763.6319115323854, 1708.7614533965245], [2490.6491751328176, 1409.8169447292385], [2396.2682881544747, 1275.0867991088005], [2432.21515434986, 1161.7044901777363], [2498.353466728711, 1050.3022801302932], [2306.0319797430366, 1064.2036012379256], [1840.4546655824354, 943.9181744256963], [1763.1584920176956, 1034.797461050202], [1689.517100263081, 1125.0470622624964], [1653.424633431085, 1841.9357038123167], [1664.0555762081785, 1461.0939591078068], [1642.073492286115, 1575.6438522674148], [1508.746829626907, 1650.7266127550083], [1389.2125221280164, 1810.2206279698128], [1298.253391898487, 1904.9219131283553], [1255.722598973607, 1236.1503848973607], [1145.8151041666667, 1289.8904389880952], [1069.2505586592179, 1425.4190875232775], [1019.3097643097643, 1683.5351611351612], [971.2508177794882, 1543.6942466807773], [604.8047164295459, 1172.743519781719], [537.6352811870306, 1107.4324967942846], [355.2927923802546, 1225.5440974448209], [197.8185450153303, 1169.3126451732787], [217.65395493111942, 963.9290210747195]], [[1973.157359484246, 1736.293289748479], [1872.9079955165328, 1794.7397720904166], [1752.5078830823738, 1880.0210806023028], [1600.9075964230874, 1902.4014090868034], [1484.6096991468344, 1991.9364167040862], [1361.4330904567264, 2099.491647414253], [1215.6422930162391, 2162.8463040198776], [1919.7982933914307, 1370.2569898329702], [1801.266255805645, 1364.5571632725973], [1685.9811631309437, 1332.6661485003658], [1520.3896278986158, 1357.5116843429805], [1293.1682251393634, 1506.2659593598273], [1405.4093457943925, 1466.8655095683132], [2253.383313656041, 865.3001443001443], [2129.3075409156827, 892.5193702092397], [2018.792987654321, 911.5942716049383], [1903.994165045846, 865.0118551449476], [1749.6438305367572, 952.7287475665153], [1637.9051254089422, 895.1640312613595], [1520.1296912553153, 818.9422259197634]], [[351.48710587777344, 2137.1877189567927], [2087.2627667607203, 1898.0796010014703], [1973.137311947326, 2028.8394901045056], [1840.9627289955781, 2140.7906269740997], [1963.05390773923, 1552.973694243233], [1814.0250808753567, 1644.5492673644148], [1681.8831725202624, 1736.3442300270167], [1879.093099452591, 1125.310584568666], [1704.747927934939, 1177.4418070742051], [1559.2371043488222, 1267.443816578214]], [[1708.4069157547108, 2489.8068704267307], [1246.6498319762181, 2426.4747964327257], [1099.3285556062285, 2312.70040393511], [1035.5831540991903, 2090.186962297571], [859.4096857418172, 2004.999005315346], [781.0607002923067, 1781.9858511101436], [607.7673179955877, 1718.9061140876142], [1741.0251464683035, 1733.528470805082], [1580.408520656626, 1665.0979133743992], [1393.796670220104, 1652.9146234401455], [1181.9163232564724, 1644.8730063858568], [1071.6571096849098, 1471.4727319637502], [892.1182789022724, 1303.9963988575687], [1913.3371287746443, 876.149270027452], [1727.7586329539577, 909.3324172271081], [1542.5318844269077, 902.4931585311525], [1261.1731986406012, 895.4994231908458], [1092.6088370493137, 789.9788298570937], [904.3720534476239, 687.9237993192992]], [[1937.8194938901183, 1101.5045703839123], [1902.1283277090363, 1674.3736407949007], [1150.1850912284126, 2144.9543370084884], [713.4411492122335, 2089.209968077438]], [[2130.038075913106, 2086.9737407495822], [1471.8242845942768, 1381.045766366131], [1342.451920243591, 1392.0814261860328], [1080.0353797089895, 1508.5970699621287]], [[1955.0932817306739, 1817.7854520194503], [1107.637832263978, 1838.711274060495], [973.8531163159984, 1740.104605001985], [866.4344820776143, 1808.424311247161]], [[2309.767108386076, 1724.7053006329113], [1972.9972699696664, 1490.7615773508594], [1202.499111549852, 1381.8846989141166], [735.6771172237011, 1281.6346248644386]], [[582.1946990833001, 1827.2436229573536], [773.6927490355129, 1764.0348204570184], [1076.4032557631378, 1575.405003823883], [1152.2987732489119, 1376.7000395726157]], [[1793.0469965764084, 1641.8003423591658], [1735.3575505167362, 1644.8579361406755], [1676.9802576415884, 1647.1164722920955], [1618.1270697250493, 1649.5752696339055], [1559.6190184518796, 1653.5832631498354], [1500.944922023583, 1654.7408900722708], [1441.9828540673536, 1655.7940950330617], [1383.3601449834196, 1656.5265674404257], [1325.254073958007, 1657.1193983077405], [1267.6406828310296, 1657.292997960571], [1873.5412611757258, 1377.071920246855], [1816.7975431861805, 1383.8168905950097], [1759.051811896984, 1390.7707209602675], [1702.3175269157696, 1399.769474350855], [1644.6540028199906, 1406.2810590631364], [1586.50668438746, 1408.8358220623877], [1529.139772281115, 1413.8116215155085], [1471.0930071543964, 1413.9636125855834], [1413.280489682784, 1413.4012935016938], [1355.6905932984937, 1412.7071933599755], [1298.7042921508182, 1410.7333017152794], [1241.6891735473416, 1405.3520333359056], [1549.9446591563194, 1174.974673135433], [1494.3664737171464, 1174.3918961201502], [1439.2348382749326, 1172.6447944743936], [1384.4401157787686, 1165.8261753891888], [1328.9481668773703, 1162.822376738306], [1277.8593833677608, 1146.7730854563538]], [[1652.833320186164, 1794.1791433304409], [1656.5860566448803, 1736.4615624027388], [1658.7902385089244, 1679.2525667351129], [1663.7148926309285, 1621.3990421751892], [1663.2730044123546, 1564.3290012033694], [1667.8735810915875, 1505.67058000311], [1669.393551373033, 1447.7375038568343], [1670.1662770765156, 1389.4250428549167], [1670.4452882827031, 1331.586949783013], [1670.4609619943412, 1273.3426626902194], [1670.0462445930004, 1215.4909948879276], [1670.3527140943827, 1158.0662569920428], [1669.9865439093485, 1100.263770853006], [1409.434824128601, 1786.8943180009549], [1411.4905464698843, 1729.8362185879537], [1414.980378250591, 1672.4854215918047], [1416.7044463818656, 1615.080050725212], [1418.8612113604268, 1557.647654166013], [1419.6024676030108, 1499.616202374486], [1419.8058761804828, 1442.3099523771086], [1420.8368187825547, 1384.7818549327528], [1420.9958437892096, 1326.5165464240904], [1419.5414934441958, 1268.5855452510393], [1421.467868338558, 1211.2114420062696], [1420.5465199144826, 1153.4252118140787], [1420.5646935989078, 1096.2268894064734], [1423.912793311157, 1044.211813359705], [1418.0883326604765, 990.4951958013726], [1415.2172988881623, 934.6486265533028], [1411.2724825839139, 878.4365896136795], [1410.319014693172, 822.0874675885912]], [[1661.534306216238, 1828.0721599612996], [1663.7786326142132, 1772.0421161167512], [1669.1763734968167, 1715.5743142340643], [1673.4985243864555, 1658.9819043181112], [1680.6272459727386, 1601.3809634448576], [1687.1103707107843, 1543.994332107843], [1693.8678861788617, 1486.7083333333333], [1702.0, 1430.5895800933126], [1708.374241044323, 1373.5820431086825], [1718.530485466677, 1316.3944320883504], [1726.8239906832298, 1259.7372670807454], [1734.656896141963, 1203.644062425731], [1745.622807017544, 1148.0423976608188], [1758.8580121703853, 1090.320111186237], [1420.1673553719008, 1828.1555944055945], [1425.048467340716, 1771.6921842400898], [1430.254995024114, 1715.410548878512], [1436.6749704375247, 1658.2486401261333], [1441.4377046624045, 1601.3443006393263], [1446.8661338661339, 1544.4111273342044], [1452.9552829749243, 1486.82004502756], [1457.8017560073938, 1429.5241065927296], [1464.7496877926944, 1372.5069466125508], [1469.1785906815555, 1315.031959953793], [1475.972532983566, 1257.848159864208], [1481.716146861429, 1201.0854322937228], [1490.6043830177152, 1144.1671502748932], [1499.7025930639516, 1087.5857461024498]], [[1661.5039001560062, 1867.836193447738], [1708.738889780202, 1823.40381838601], [1695.8537439908744, 1769.9595046036013], [1646.2733425893787, 1708.3727872266575], [1628.5603031280232, 1657.5560303128022], [1607.504141815772, 1606.969350563287], [1582.6567623803578, 1559.6357053682896], [1560.3010456350814, 1510.7811461457404], [1536.0794381398302, 1461.1672883104516], [1512.5661606995545, 1411.7915360501568], [1490.11886976291, 1362.0360506658005], [1465.1065580724528, 1314.6926274709426], [1440.4799608993158, 1269.0662045676709], [1470.308085017712, 1905.4752031673265], [1455.0770911360798, 1856.3770287141074], [1413.2226855713095, 1812.8654434250764], [1394.6307735902521, 1766.930102956456], [1377.527967806841, 1718.655050301811], [1352.596145927695, 1669.5880754344066], [1328.2601242236026, 1622.071055900621], [1305.4256108378356, 1572.567131682929], [1284.4658538664848, 1522.8210418620513], [1261.1681669394436, 1474.1680032733225], [1240.5895291461068, 1423.9354353098604], [1218.4440958815958, 1374.584942084942], [1199.446684219187, 1322.8948494322856]], [[1432.78234375, 2066.80796875], [1450.4901239111766, 1984.4523371365476], [1246.6730103806228, 1985.0882352941176], [1305.061249580865, 2167.8726947580194], [1598.431036577956, 1968.0695102685625], [2475.1968137254903, 2004.6327205882353], [2261.7126861493675, 1956.6400179151271], [2378.7159016033384, 1915.7601581374918], [2378.576618310374, 1836.9714738510302], [2409.715123354353, 1760.892576612457], [2277.196277676757, 1730.9716547224302], [1583.8043715846995, 1377.9010928961748], [1334.3320195195195, 1157.9759759759759], [1183.4126984126983, 732.4336860670194], [1151.5170414617007, 658.7450808151792], [1144.7930542340628, 561.2616555661275], [1104.8655862726407, 705.2793136320305], [1064.4471981576928, 768.0686478780568], [874.3258658132359, 790.8679849125614]], [[1152.1021148036255, 1541.1256797583083], [1116.3264218695701, 1496.3187767867485], [1199.4474891214866, 1447.0076443608139], [1221.26122764123, 1376.0235490927807], [1115.5303809147226, 1591.0123896138132], [1220.4538671088476, 1552.4153736823405], [1052.4523666866387, 1441.211703614939], [1079.9808839779005, 1365.5], [1094.3424424888065, 1293.4139262004014], [2547.556857339766, 1030.9107512060648], [2061.4656660412757, 1075.2357723577236], [2049.9730841121495, 1000.8439252336449], [2126.0623292773776, 994.5127886764341], [2018.8510790509122, 926.0268272326219], [2109.8660684240044, 886.5061132922042], [2206.889635073492, 890.4883426254435], [1020.373106060606, 604.6076203208556], [969.0165708488332, 510.5040018036298], [1200.084813338166, 530.3503684909991], [1075.953515949247, 459.1551049448595], [1225.099222198865, 417.79083455959636], [1174.1614065742056, 329.9148192639511], [1093.4226666666666, 266.58386666666667]], [[1910.807038355081, 2297.468828258864], [1940.2744399185335, 1745.8379582484724], [1943.997855701312, 1660.9481584258324], [1949.0764667956157, 1575.8389426176661], [1953.153187683659, 1489.2939823687236], [1956.054085303186, 1404.005781089414], [1956.522412286178, 1316.8045948308154], [1955.7003277861825, 1231.713313161876], [1954.9400968646444, 1144.781162375733], [1954.2514450867052, 1060.136089469716]], [[1863.3652037617555, 1889.2084639498432], [1666.1529051987768, 1915.0565749235475], [1727.651924375422, 1814.2383524645509], [1644.2597669636737, 1837.3245373543523], [1561.7879918588874, 1859.851763907734], [1481.8779434850862, 1881.2868916797488], [1416.9140866873065, 1914.267801857585], [1361.4374768089053, 1920.347680890538], [1584.8076923076924, 2003.0805288461538], [1516.4991487912837, 2058.787878787879], [1440.0958453311393, 2079.546688605512], [1371.1787175989086, 2091.6283765347885], [1211.9486772486773, 2120.7428571428572], [1643.0396341463415, 2196.4776422764226], [1556.7657438393671, 2042.021904472163], [1478.2102079395086, 2074.2264650283555], [1410.3547826086956, 2087.4695652173914], [1327.2390619530977, 2105.8613930696533], [1175.082105263158, 2141.6142105263157], [1143.3858138581386, 1925.40016400164], [1068.384770889488, 1896.3648921832885], [995.732967032967, 1858.0582417582418], [918.7938638542665, 1834.0540108660914], [853.25, 1808.0391969407265], [985.2174810736408, 1990.8860977288368], [915.3287853577372, 1942.0522462562396], [831.3274364747507, 1910.7143776133805], [746.2874801901743, 1896.6291600633915], [1107.0515260912373, 1913.0259271414507], [1030.8830780724693, 1878.7583115427717], [955.8759924385633, 1850.3077504725898], [883.5067650676507, 1818.190241902419], [816.2502463054187, 1805.3822660098522], [1017.9338301043219, 2022.7698956780923], [952.0833601544898, 1965.7901512713229], [878.0207219251337, 1923.8495989304813], [788.9736495388669, 1902.7878787878788], [703.4099157485418, 1891.0123136746597], [1340.3807971014494, 2373.58731884058], [1259.5956090651557, 2358.3240084985837], [1103.8750517170045, 2331.150186181216], [1025.6749285033366, 2323.113123609787], [944.7290926703262, 2310.9247659024863], [882.1028131279303, 2298.3030810448763], [801.4208609271523, 2275.7844370860926], [724.2593155893536, 2248.912927756654], [671.9671568627451, 2229.679411764706], [612.625314861461, 2204.5236146095717], [541.851086569291, 2161.745279657998], [476.28665785997356, 2113.2093791281372], [416.4384882375627, 2063.4785962205938], [1176.0509373894588, 2345.070038910506], [1374.2085751128304, 2395.7520954223082], [1298.4942568743475, 2361.541942220675], [1218.7392532161907, 2351.6319422654533], [1138.212015503876, 2338.755426356589], [1067.2086740135087, 2328.145040881621], [985.10426716141, 2316.4022263450834], [912.5312710154674, 2305.9892400806993], [841.6205507051712, 2287.8176628609804], [761.4405856315968, 2262.5069799114744], [695.8093730208993, 2239.5142495250157], [647.4030335861322, 2213.6170097508125], [572.1432858214554, 2186.61327831958], [509.15290102389076, 2138.390784982935], [444.42912280701756, 2088.0491228070177], [388.6301232922359, 2036.5408197267577], [1393.0, 1913.0561630218688], [1446.4200654956833, 1901.4715689193213], [1521.1662935721138, 1870.6984330028781], [1602.8378623188405, 1846.7629830917874], [1687.042180402336, 1826.5681375730046], [1874.824512534819, 1917.206824512535], [1864.521359223301, 2030.1126213592233], [1504.1270557884554, 1568.6439858110286], [1585.2062822315986, 1573.9174871073606], [1654.576674600476, 1577.6208772526352], [1738.6328279499012, 1556.479235332894], [1813.1336687028386, 1532.104922745239], [1872.8231448185682, 1504.9568486433475], [1782.0761817347407, 1624.3180357962367], [2054.4489795918366, 1391.7562358276643], [2328.7801242236023, 1318.9652173913043], [2359.712273641851, 879.6941649899396], [2403.9102564102564, 802.8177655677656], [1224.0482985729966, 1474.0965971459934], [1212.739780658026, 1604.734297108674], [1268.4116161616162, 1664.1346801346801], [1286.0639496355202, 1726.6660039761432], [1548.716056670602, 1572.5073789846517], [1614.7821475916924, 1568.6098099867434], [1696.8336880510794, 1568.7965947096382], [1777.268387553041, 1545.1251768033947], [1842.0888157894738, 1518.205043859649], [1767.81669535284, 1675.631669535284], [1902.6627272727274, 2028.5827272727272], [2047.9832746478874, 1353.3301056338028], [2372.9185897435896, 833.7846153846153], [1242.7638078132015, 1639.1158509205209], [1285.629129129129, 1684.8521021021022], [1258.7434743474348, 1493.4599459945994], [1678.2961165048544, 1624.6291262135921], [2695.4521327014218, 1299.5308056872038], [2706.325708602685, 1334.4599701640975], [1719.5741758241759, 1614.5]], [[1323.8405197873597, 899.1754282339043], [1378.2160702228225, 912.8338960162052], [1430.9524375743163, 932.4155766944115], [1482.5074626865671, 954.2719013627515], [1534.5705045278137, 977.7050452781372], [1578.659829059829, 1003.4564102564102], [1621.3689740420273, 1025.7867737948084], [1666.1108786610878, 1052.692468619247], [1709.175572519084, 1085.0896946564885], [1749.4234567901235, 1122.0456790123458], [1786.9228840125393, 1163.9485893416927], [1395.77048070413, 857.8767772511849], [1447.4663494221618, 868.6668932698844], [1499.8265245707519, 889.0556542332741], [1551.8832612723904, 910.6893143915997], [1603.5, 936.2579250720461], [1651.5365448504983, 956.6901993355482], [1689.1107614852108, 986.6784140969163], [1731.3383177570092, 1022.5781931464174], [1774.1871382636657, 1062.3967845659165], [1815.1242973141786, 1103.5159275452843], [1850.7106842737094, 1148.171668667467], [1319.8685714285714, 934.9371428571428], [1363.970080552359, 968.7209436133487], [1417.6398843930635, 989.164161849711], [1421.7859649122806, 862.240701754386], [1473.2044303797468, 877.4613924050633], [1526.36503683858, 900.0147354320161], [1578.1116898148148, 922.8587962962963], [1628.8551549652118, 946.5660974067047], [1670.9310897435898, 968.09375], [1710.918956043956, 1003.8722527472528], [1752.937235968618, 1042.34821967411], [1794.3085311096856, 1082.4772289929442], [1831.985254691689, 1126.0187667560322], [1879.5551102204408, 1157.5591182364728], [1470.947003154574, 1015.7753943217666], [1520.7842794759824, 1040.3432314410481], [1568.3359173126614, 1070.7086563307494], [1617.5829411764705, 1102.4876470588235], [1665.1230677764565, 1136.5957193816885], [1710.9538188277088, 1170.6252220248668], [1658.610086100861, 1193.6445264452645], [1609.9648682559598, 1172.9642409033877], [1558.6474474474473, 1142.314114114114], [1508.835076923077, 1113.3987692307692], [1459.610219845514, 1087.3232323232323], [1409.8588382261087, 1056.2279825109306], [1295.6408544726303, 893.0353805073431], [1351.6532697547684, 904.9543596730246], [1403.8223684210527, 923.1881578947368], [1457.811320754717, 942.7561146051712], [1508.9327731092437, 965.2857142857143], [1559.2298136645963, 989.4147688060732], [1597.7984886649874, 1014.2636439966415], [1644.3881824981302, 1039.632759910247], [1687.1281186783547, 1068.3897505057316], [1729.5906137184115, 1102.917689530686], [1769.447033898305, 1141.8255649717514], [1806.1818181818182, 1185.1949934123847], [1663.0304798962386, 1217.7788586251622], [1636.6254801536493, 1185.2784891165172], [1584.6404293381038, 1157.4800238521168], [1533.4151436031332, 1128.2330287206266], [1482.1079136690648, 1103.6625899280575], [1434.0730198019803, 1071.66150990099], [1386.7831753554503, 1040.677132701422], [1733.023091725465, 1190.147530468249], [1689.277700831025, 1151.777700831025], [1642.6448160535117, 1117.5297658862876], [1592.561862244898, 1085.4955357142858], [1543.889913899139, 1055.0276752767527], [1497.4845971563982, 1028.7618483412323], [1444.8781869688385, 1001.4300283286119], [1390.6439649359406, 979.1051921780175], [1339.909574468085, 942.3936170212766], [1665.3684534812076, 1286.1583487369069], [1618.8032589016295, 1253.7585998793], [1570.198192382182, 1221.2569399612653], [1521.3708381171068, 1191.1199770378876], [1471.1540243196293, 1159.6902142443544], [1418.3955056179775, 1128.8550561797754], [1341.5049151027704, 1083.6300268096516], [1281.6609724047307, 1043.0637319316688], [1244.9877675840978, 1007.7902140672783], [1213.21875, 962.8722222222223], [1686.972386587771, 1321.0729783037475], [1641.9547602970965, 1270.1478730587442], [1594.0751086281812, 1237.4121663563005], [1546.3862467866325, 1206.0841902313625], [1495.3837705918243, 1176.2239170225748], [1443.8626342566422, 1145.9892594686264], [1392.6981566820277, 1114.6336405529953], [1342.7750294464076, 1083.2014134275619], [1294.4664031620553, 1060.0065876152833], [1260.7695439739414, 1028.571661237785], [1228.757742601514, 984.6441844459738], [1196.4718826405867, 940.8300733496333], [1514.7855504587155, 826.7769495412844], [1548.5808695652174, 867.5513043478261], [1599.5348837209303, 869.5565891472868], [1649.339824120603, 893.5866834170854], [1700.2267587939698, 918.7261306532663], [1747.1917424772569, 946.8173547935619], [1789.0977804439112, 978.6316736652669], [1830.7698561601, 1015.9118198874296], [1873.0962962962963, 1055.361111111111], [1911.7368746139593, 1095.2452130945028], [1935.5311143270621, 1166.7677279305356], [1891.7641456582633, 1204.4761904761904], [1845.7479623824452, 1244.889028213166], [1794.3449139280126, 1291.5405320813773], [1715.405897114178, 1328.7358845671267], [1652.9615137908916, 1361.3181526619628], [1605.3609427609429, 1388.0760942760942], [1552.1704611792177, 1410.8727378867484], [1534.0848729792149, 1266.7430715935334], [1537.4084507042253, 838.6539235412475], [1580.0097276264592, 860.8317120622568], [1623.4671215880894, 880.7853598014889], [1673.9619600257897, 907.6673114119923], [1724.123203285421, 934.0499657768652], [1768.0847583643122, 960.8869888475837], [1810.429873772791, 997.2678821879383], [1851.871924290221, 1035.6132492113566], [1892.1113430758526, 1075.5539318023662], [1937.4584500466854, 1110.6825396825398], [1914.2722891566266, 1184.0981927710843], [1868.9289838337183, 1226.236143187067], [1829.3644388398486, 1260.156368221942], [1795.1859131859133, 1292.6224406224405], [1742.6694695989652, 1322.6869340232859], [1678.88052271313, 1348.6278780336029], [1629.073598919649, 1375.3855503038487], [1579.7866915422885, 1399.4023631840796], [1524.6983094928478, 1420.832899869961], [1558.2978177150192, 1283.6142490372272], [1509.0995115995115, 1249.8925518925519], [1483.997975708502, 1235.4912280701753], [1459.8938356164383, 1221.5863013698631], [1437.1553672316384, 1207.4449152542372], [1413.3222871994801, 1194.4269005847952], [1389.3242692046226, 1180.1264445955132], [1366.4498869630745, 1168.3775433308215], [1344.1847968545217, 1155.7116644823068], [1316.207624234173, 1145.5302927161333], [1288.4657701711492, 1139.5409535452322], [1261.0919298245615, 1134.4715789473685], [1232.4394865525671, 1130.4755501222494], [1206.4891799544419, 1190.0466970387245], [1202.9459287531806, 1126.2754452926208], [1174.8713826366559, 1122.1967845659165], [1148.3100775193798, 1116.937279774489], [1122.4837570621469, 1113.3629943502824], [1096.491571139582, 1108.2440997977073], [1068.349196538937, 1103.2713226205192], [1875.32664756447, 1868.1432664756446], [1913.1995332555425, 1837.1610268378063], [1956.6508538899432, 1805.1290322580646], [1650.9323770491803, 1854.125], [1637.2044198895028, 1825.2495395948436], [1341.9438405797102, 1949.4375], [1240.6431818181818, 1647.9363636363637], [1207.3072289156626, 1649.9558232931727], [1155.5058139534883, 1564.1899224806202], [943.8494101318529, 1220.6904927133935], [951.5278436018957, 1193.2298578199052], [958.3406658739596, 1163.7717003567182], [961.0844564240791, 1137.4429469901168], [973.2523076923077, 1115.096], [989.784398034398, 1090.4367321867321], [1004.8660377358491, 1065.354716981132], [1021.9539198084979, 1041.6104129263913], [1038.1456908344733, 1017.7927496580028], [1056.3811718235681, 996.9177090190915], [1076.533244680851, 976.7253989361702], [1097.2739130434782, 957.2354037267081], [1119.7772727272727, 939.4525974025973], [1142.4679127725856, 921.6442367601246], [1166.5814382298709, 903.5107559926245], [1189.8962386511025, 885.7334630350194], [1212.8909722222222, 870.4354166666667], [1237.0153846153846, 856.3929487179487], [1262.5938543754175, 844.685370741483], [1288.6099092812283, 832.8876482903], [1576.1944700460829, 800.8202764976959], [1354.2884615384614, 696.3568376068376], [1346.2948490230906, 662.5050325636471], [1254.9031007751937, 710.8023255813954], [1014.7700258397933, 628.2230835486649]], [[958.9634146341464, 2442.060975609756], [952.6826923076923, 2301.1346153846152], [939.688524590164, 2173.44262295082], [932.9276315789474, 2025.717105263158], [910.658064516129, 1889.8], [867.5038759689922, 1759.3565891472867], [768.4963503649635, 1653.094890510949], [959.3303571428571, 2433.2946428571427], [946.3701298701299, 2290.6428571428573], [946.5138888888889, 2181.8194444444443], [932.7094017094017, 2027.7692307692307], [907.6897810218978, 1892.985401459854], [871.4187725631768, 1767.1913357400722], [788.6369863013699, 1662.8561643835617], [950.070775924583, 2445.5032632342277], [946.3089331912951, 2307.548166184751], [943.8750722125939, 2169.669410745234], [895.0, 2414.0], [933.3828228052176, 2029.8529972153012], [997.0, 2300.0], [907.6087877931619, 1893.1011585193557], [938.25, 2395.5], [866.2018072289156, 1759.0180722891566], [949.9, 2411.9], [989.75, 2336.75], [893.5, 1932.0], [783.1181925476311, 1664.1682684201653], [934.7222222222222, 2207.6666666666665], [950.0872444572416, 2515.057875035992], [933.4285714285714, 2087.714285714286], [928.0434782608696, 2017.7391304347825], [947.4779892076115, 2373.8074410678782], [821.0, 1668.0], [945.5127272727273, 2239.8598484848485], [898.0, 1859.45], [884.7647058823529, 1792.8235294117646], [939.6279507603186, 2099.3358435916], [922.0946246112838, 1960.8699837109432], [805.9714285714285, 1691.2857142857142], [753.5625, 1691.6875], [889.0927632572897, 1825.6952711890015], [842.3767959770115, 1696.8816451149426], [713.3099573257468, 1682.0129445234709], [1537.6741914191418, 2511.186402640264], [1396.8713876967097, 2542.149785407725], [1261.446687517771, 2574.381433039522], [1125.637513496838, 2604.9018972697827], [1796.242518351214, 2710.7556465273856], [1764.5713083718974, 2574.4819800869445], [1733.1865661596446, 2437.202449858662], [1697.2771723945236, 2301.6241967029896], [1656.4882648784576, 2166.585079631182], [1478.4356253537069, 1668.374221844935], [1433.4602486267706, 1535.2327262214512], [1385.1895773492, 1399.9235398714266], [1327.611711838225, 1270.8611150119366], [1265.6122304199773, 1144.1794835414303], [1199.1617688217184, 1018.6540722082284], [1112.3116163016357, 904.2217909620183], [1029.4296361245067, 789.5895075259389], [1465.9583821805393, 2529.1398007034], [1328.3428613525857, 2556.8634153528806], [1192.1692829868477, 2590.238155847829], [1059.5933371625504, 2621.2948018380243], [1780.689030575279, 2642.932328648022], [1748.5157543197329, 2505.672135908233], [1716.0711702589324, 2368.6638217850427], [1678.4810419681621, 2233.8557163531113], [1634.1009403467529, 2100.419335880106], [1482.421458625526, 1736.02688172043], [1455.881810319977, 1601.228307869703], [1410.206643598616, 1468.0034602076125], [1357.0575098525762, 1334.759305210918], [1264.5742768885145, 1142.8960966020782], [1162.1469544228312, 958.0269771404231], [1072.1379796839728, 845.5437358916479], [987.851458031255, 738.8416304172708], [2056.9637188208617, 810.9591836734694], [2018.3366565525214, 683.7569715359052], [1940.4730623153748, 566.9554086369391], [1859.3280408670441, 451.36448985227116], [1775.2964768138143, 335.48586547834566], [1689.2898275358082, 223.68781058170126], [2044.5096592904813, 291.75693712680015], [2039.8545375597048, 748.0067303517152], [1979.54696055033, 625.6066264214517], [1900.841798501249, 509.075492645018], [1817.7214324767633, 391.9923455440131], [1732.9158734602463, 278.91433370660695], [1635.4409418913785, 191.99012533232056]], [[1781.6090230361815, 719.6555420841044], [1789.188293502614, 1265.0306198655712], [1924.8477429849531, 1837.1259455063034], [870.8114623607005, 1818.872071867182], [942.7422927267286, 1363.7460341215206], [1048.3743492190629, 881.7140568682419], [1801.8775294847921, 843.830540037244], [1902.5006534727409, 1321.4699402539209], [1708.895746628933, 2038.515606792076], [985.3617265526347, 1807.1012512914706], [1013.3656622724419, 1311.262554927809], [957.7488789237668, 808.0831091180867]], [[1544.5491507977354, 1177.7538600102935], [2200.1454545454544, 1076.9182472204054], [2070.1300715990456, 1065.1774065234686], [1921.2090897045184, 1068.521929243408], [1639.289544235925, 1053.0280863015448], [1489.5040508537954, 1143.2046615979061], [1655.7901003344482, 1238.4235451505017], [1482.225890985325, 1528.6105870020965], [2010.8578347951302, 1400.261814373609], [1867.028541908821, 1414.822420250549], [1935.3417326613871, 1583.6532277417807], [1777.6170788253478, 1651.5855229263266], [1821.8769032258065, 1789.3508387096774], [1788.9625030735185, 1957.457585443816], [2284.8681254388016, 1983.0521881582026], [2180.254087754382, 2055.1743324314784], [2080.201543522267, 2136.9194078947367], [2113.868440145103, 2292.4368802902054], [2265.7118232890703, 1072.5740551583249], [2133.673217368211, 1060.7761253485594], [1983.7650273224044, 1085.36179946626], [1860.1910022627446, 1052.980300811926], [1697.2078919491526, 1067.994968220339], [1577.758662207358, 1039.8128428093646], [1600.1900334696913, 1209.8738068674847], [1711.5289560579122, 1261.707772415545], [2076.1731137495212, 1402.2209881271544], [1929.5438119820676, 1401.7994837657927], [1806.0702548842023, 1423.7755207659463], [1875.4656024862268, 1573.1741771436643], [2004.82156133829, 1588.248274030802], [2231.542984060622, 2017.4025346224196], [2131.2875569044004, 2096.7287556904403], [1825.470853858785, 2010.3674055829229], [1757.0276065398016, 1788.611766282498], [1728.6144758283542, 1618.4197447039653], [1535.1775348661893, 1565.8988566402813], [1431.7490977335065, 1494.6829796448678]], [[1612.1488270411166, 1269.151433616413], [2169.462575351641, 1509.0451272605492], [1969.2456950860983, 1509.7196136077278], [1584.2035714285714, 1358.1188775510204], [1423.8265488229795, 1432.6513979773945], [1434.2970388390236, 1147.3619662779968], [586.7070256625999, 1340.9840976020193], [399.6743508441169, 1374.7229411260605], [2263.1688235785255, 1592.4441846873174], [2132.571718710333, 1383.6338326140306], [1938.8511984021304, 1239.6777629826897], [1904.236730540953, 1370.8388163472953], [1767.6514058223438, 1532.143236294269], [1723.0608963678774, 1274.6896867710764], [1562.9796271857472, 1031.3376773342131], [1352.0696191914108, 1063.143432310015], [1464.4878994856228, 1275.9081710093599], [1333.584368398817, 1422.0313476975075], [525.6951835634894, 1260.6155271135062], [329.4216145191755, 1312.6876899072022]], [[1723.284634115084, 1849.4543124439317], [1613.8301983365322, 1856.229558541267], [1503.8146046159668, 1856.4341026611173], [1393.5417006882071, 1845.4631984136242], [1287.2065709969788, 1826.9962235649548], [1297.949938347719, 1626.317879161529], [1210.1037735849056, 1687.455768635942], [1370.9466201716739, 1223.9936963519312], [1269.458962090541, 1268.7037173352962], [1165.3138403990024, 1306.3903990024937], [1058.692271497471, 1335.5636983884249], [1441.2748138200463, 1045.9025759980466], [1330.1736394359632, 1029.2540496445636], [1209.9181894183685, 1012.480950674788], [1101.5366092164772, 990.5182740496272], [991.843312686396, 965.7604955264969], [884.1964117091596, 939.8825542965061], [1775.8989123480487, 1842.8476007677543], [1668.6422029404675, 1853.0416967944084], [1558.9328269641585, 1854.0525753619747], [1449.81853043048, 1850.5499752597723], [1339.3749190309625, 1836.05415209224], [1343.9374285060626, 1593.4704873026767], [1253.9808384624318, 1661.7553129717803], [1416.1071472964575, 1198.0524549409572], [1321.134958590925, 1246.893269567246], [1218.2864890924432, 1290.5170543569966], [1113.029046673287, 1322.1436196623636], [1005.2988341968912, 1347.6349740932642], [1386.0605616605617, 1038.7150183150184], [1265.2493814943098, 1017.8984413656606], [1154.9590971442824, 1002.4287317620651], [1046.9869145163264, 980.4784150666503], [937.1605346912794, 955.1530235518778], [801.4288432267884, 925.8509322678843]], [[2407.3852459016393, 1053.9452363981943], [2221.3570179475378, 2119.1080533824206], [967.6558441558442, 1969.8841991341992], [899.6656015559878, 1168.2771603223118], [2334.530429594272, 976.9883054892601], [2442.4286559394227, 1146.4564600094652], [2156.795495072736, 1952.6357578601596], [1069.4427986132998, 1929.2289631263789], [849.4241452991453, 2047.6221001221002], [1001.3537643436888, 1191.466694654352], [818.5968373915478, 1144.200391827596], [922.2002915688022, 865.6557058394752]], [[741.110183639399, 1986.117696160267], [2566.579543990725, 820.0413242303233], [2457.979235880399, 580.9754983388705], [1713.0847533632286, 445.227466367713], [2761.3254950495048, 1061.118193069307], [2689.6792390405294, 926.5371381306866], [2424.9783248814, 679.9442990348438], [1741.6263921385118, 398.9153018249883], [1696.5733748028736, 533.4100227790433], [1981.235606261608, 980.2128946670205], [2029.936065500425, 1433.4054852131896], [1815.3388654836376, 1521.5019362526066], [1826.457278608977, 2426.9040841083706], [1258.6250662627792, 2174.659485043544], [1218.0437832019722, 1905.0581229818408], [1967.10239906378, 231.41632533645407], [2369.744939271255, 1239.8427006430102], [2114.880912386173, 1420.580143223411], [1917.937833118912, 1474.6851222201801], [1736.7339183577528, 1563.9182794013946], [1981.9035687331536, 1830.0506307277628], [1925.292768457966, 2119.2813477395507], [2045.4785061491593, 2300.173417578698], [2208.5092441310517, 2510.116605039126], [1892.9637913329225, 2630.338899157938], [1287.1083734716563, 2304.569507224898], [1235.8336285145008, 2041.7398715961922], [1209.976788925599, 1775.0670402743508]], [[2151.0790934844194, 2500.569971671388], [2524.189361702128, 1700.4615930169123], [2522.7667455621304, 1220.9979881656805], [2407.274429967427, 566.1700791065612], [1895.0309980610266, 935.9495611797122], [2005.4139937553223, 1553.741413567982], [2185.653087004579, 1978.5994315490288], [2197.687359812278, 2399.5840091100454], [2379.617572276109, 1670.5728411338168], [2456.1913337527526, 1489.4228530984587], [2365.450547673902, 1160.8859787397437], [2425.60722517522, 764.0127005033163], [1969.6698728579327, 809.6773908236595], [2034.4845766337726, 1364.1505532169392], [2138.7738549972187, 1823.4355275356945], [2095.803019250896, 2246.3866988043515], [1955.3206266318539, 2565.8637075718016], [1690.2252167125425, 2255.2626432767756], [1466.3340782122905, 2544.2201117318436], [1538.2684348430564, 2368.2954922196673], [973.1654759606101, 1128.430971229967], [1149.4995618713806, 1188.2924794270039], [1082.4462719666678, 752.303904438695], [605.0890001990314, 673.7307934717708], [968.4263646394496, 2206.659521646005], [848.1187807570683, 1708.889190352338], [401.2010744664975, 2294.5706014027755], [430.3231622964221, 1741.2021473935736], [505.8669773251345, 1193.3589066102998]], [[2228.1744779998207, 2119.8139618245364], [2241.6257976299, 1595.0611668185961], [2147.6781484495073, 1657.1849742337945], [2008.1170577868472, 614.7043873481124], [1827.7363505747126, 606.1611889367816], [1134.4186569261028, 645.7745153132903], [1007.9316203313676, 713.9959281100814], [1071.5389129523544, 1076.1193237389969], [1251.9930177349531, 1288.4778662198016], [1050.0543148607665, 2313.500964984836], [1108.3387604323436, 2493.2713093446437]], [[572.568855788761, 1622.2487474610698], [2200.5543563068923, 1760.0500650195058], [2012.8936400541272, 1799.0300405953992], [2308.5808531065695, 2097.4132492113563], [2045.8694437050838, 2084.2351610327387], [1336.639119601329, 1848.5005537098561], [1160.1937436932392, 2022.4251117197637], [686.9901241534989, 1209.0795711060948], [605.7104887110603, 1713.3975421549014], [643.5164198325821, 1930.2561171925306], [1351.3654494382022, 1943.945926966292], [1152.772859719717, 1938.4341612321355], [1205.5397653194264, 2103.824858757062], [2237.0037599033167, 2029.5453202631932], [2305.665929814866, 1800.8998342083448], [2103.8355834781446, 1781.9465312124048], [1967.0399625768512, 2000.0116279069769], [1826.914978094195, 823.7751916757941], [1467.5331094049905, 670.3164587332054], [629.2839921502664, 1146.3816932996915], [593.718731208659, 1263.679945880938], [652.1225567127979, 1834.5818233699529], [558.8055906728473, 1874.418036241045], [449.6731691717301, 1739.6339942436841]], [[1247.6861246541046, 1011.2808011595731], [984.3616356296393, 985.1407735382211], [2312.112657388999, 1705.7907223326706], [2114.247661594304, 1991.200055842524], [2161.2223537146615, 1627.224852071006], [2029.1439322671683, 1767.1829055234512], [1903.9798197228574, 1914.361899636755], [1898.4452634452634, 1502.1422331422332], [1784.4615997874037, 1657.9808663300557], [1198.8175, 929.7227631578947], [995.1795504385965, 1076.7945449561403], [878.3497169681071, 941.6647797873809], [2333.8712004121585, 1625.2485832045338], [2274.9712112982074, 1793.5935632808257], [2207.1305149479786, 1980.1405241669959], [2020.7507304116866, 2006.0172642762284], [2229.631941734946, 1558.6289504486929], [2094.774420421985, 1697.3559520708518], [1964.386125480833, 1838.1737630985542], [1848.2792743265531, 1990.7431280923583], [1951.741776989218, 1424.1624129930394], [1842.628864623076, 1581.2510196026838], [1725.758712015612, 1731.2518817953721], [1319.832310838446, 749.2204206836109], [1321.4786590351594, 973.3362224039248], [1156.982662784542, 1028.113155108523], [944.4428175702413, 898.1910038253528], [902.1378179910831, 1049.5824809861]], [[1290.8743799603174, 1925.477802579365], [954.2033538936006, 1702.2590593677717], [1836.772130555221, 779.562326869806], [1990.7961262553802, 920.5278335724533], [2120.7001400025456, 1055.8659793814434], [1977.4747474747476, 1048.4219576719577], [1796.7600741656365, 957.7077873918417], [1597.0077386070507, 801.7513818941162], [1607.6192553810354, 666.6461605584642], [1487.4525034508722, 854.503199899611], [1456.6402636309167, 1060.2051527860995], [1264.2255667990264, 1814.627257589343], [945.8983397190294, 1794.0837803320562], [924.4519730998604, 2084.1745971323435], [738.1319507748404, 1936.1152005469462], [1908.6779850746268, 845.8951492537313], [2054.6864259028644, 985.3288916562889], [2062.058542413381, 1113.4668458781362], [1889.0982666015625, 996.382080078125], [1704.8830808080809, 922.4713383838384], [1529.3971992361553, 717.4912794398472], [1483.2636882360082, 963.4247905790943], [1419.1832051751496, 1153.7278164286586], [1206.1457038391225, 1905.5100548446069], [896.1963137591084, 2181.72667523932], [912.110854206808, 1963.9353885677585], [940.5226715686274, 1449.4114583333333], [854.2176252081465, 1731.3295760215192], [796.9606412094357, 1851.1498761892349], [674.8990550472477, 2011.020398980051]], [[1502.9465002229158, 1787.1135086937138], [1902.1960563867508, 1375.06823488714], [1913.0744559993009, 1512.2822686358472], [1450.98106549617, 1711.6880110164386], [1341.9756818373723, 1580.8225111880436]], [[611.8440814952012, 1646.8790031991919], [442.80209801671856, 1555.5544009178823], [2311.0720735565515, 2176.0975810703926], [2307.971636178214, 883.5303715979348], [2158.832609268036, 2246.42074776198], [1792.8759658939516, 1972.7568611777244], [558.9885564548229, 1448.969318031047], [820.2418860166492, 715.4433622055205]], [[2321.317805700276, 2288.6104811523137], [2250.496337532962, 2311.4479929680633], [2180.9338407494147, 2335.970725995316], [2111.709533709534, 2362.6073656073654], [2045.1427261613692, 2393.131723716381], [1864.207796193984, 2037.3247391037446], [1797.2554878048782, 2011.0231707317073], [1727.1981452859352, 1985.584544049459], [1658.099120945741, 1962.0394058805698], [1587.0915795943454, 1941.8647818070067], [1517.5054140127388, 2024.041719745223], [1480.0173410404625, 2086.527228475814], [1434.8250307503074, 2152.367466174662], [1249.134657151722, 2130.744647843624], [1184.16, 2165.07104], [1106.4211035818005, 2197.153920619555], [2110.222805933251, 1097.662237330037], [2035.1239616613418, 1072.8127795527157], [1964.1460317460317, 1036.0447619047618], [1861.975534221121, 820.5270981728089], [1791.7353333333333, 846.9036666666667], [1656.8980294025648, 825.1154207069127], [2286.0618031220133, 2300.435170436445], [2216.2815210140093, 2323.767178118746], [2146.4757834757834, 2350.525799303577], [2078.588105726872, 2378.730648206419], [2013.052801030264, 2409.3023180940118], [1898.5025094102887, 2050.6019447929734], [1830.7045080618625, 2024.3158933859822], [1762.0975458216838, 1998.3510406958683], [1692.5247035573123, 1974.826416337286], [1622.7007538511964, 1951.7050147492625], [1536.3149390243902, 1992.9368902439023], [1498.6286549707602, 2054.794996751137], [1448.9428715874622, 2115.8354398382203], [1413.9179487179488, 2181.6759615384617], [1216.397041062802, 2148.0123792270533], [1149.8255076142132, 2181.576459390863], [1071.7954116907606, 2203.411376492772], [1565.1173598906464, 1608.0970513571567], [2075.314578005115, 1087.6432225063938], [1997.370728562218, 1054.560606060606], [1932.276721720162, 1017.9171081333749], [1828.3649386084585, 832.6582537517053], [1758.4773836641718, 862.0533680442564], [1635.9036262659263, 853.8778177066318], [1450.183595001602, 1073.1140660044857], [1486.3755910165485, 1069.1010638297873], [1408.7381796690308, 1089.6903073286053], [1231.8247143716176, 1157.186410102225], [1471.0386488131467, 735.0042604990871], [1399.0174278846155, 722.9191706730769], [1325.7263652378156, 714.3044627128596], [1251.977211394303, 712.0668665667166], [1435.3470999088977, 727.7251746128151], [1362.394128113879, 718.9715302491103], [1288.9417360285374, 712.7360285374554], [1214.679329943165, 712.9545318576129], [1108.2627543497492, 821.7095252138013], [1004.5904264680003, 818.3638763418784], [960.5751085383503, 816.1646888567294], [885.8012170385396, 816.485946102579], [812.8551020408163, 822.0565597667638], [1145.2391433670434, 824.5232004759072], [1071.6693423896265, 820.5121951219512], [997.5740682743501, 818.8114625743815], [923.4612601748568, 815.0512511305396], [849.3375462392108, 818.8193588162762], [776.2622901678657, 827.2353117505995]]]
    centers = np.array(centers, dtype=object)

    # initalize counts
    number_correct = 0

    # open predicted segmentation
    prediction = imread(prediction_filepath+ str(imagen0)+ '.png')
    prediction = prediction[:,:,0]

    # get centers of sections in the prediction
    centers_prediction = get_centers(prediction)

    # get centers of actual sections and create an array from those
    centers_actual = np.array(centers[imagen0])

    # resize coordinates of centers from 2944x2944 to 1024x1024 pixels
    coords = centers_actual
    coords[:] = [coord / 2944*1024 for coord in coords]
    coords = np.array(coords)

    # loop over all sections found:
    for i in range(len(centers_prediction)):
        # get coordinates of section
        coord_prediction = centers_prediction[i]

        # find closest actual section
        closest = find_nearest_vector(coords,coord_prediction)

        # determine distance to closest section
        diff = np.linalg.norm(coord_prediction-closest)
        if (diff<5):
#             print('correct!')
            number_correct += 1
        else:
#             print('incorrect!')
            pass

    return number_correct, len(centers_prediction) - number_correct, len(centers_actual) - number_correct

In [17]:
filepath = '../Random Mess/Final/compare/felzenzsedge/'

interact(compare, imagen0 = (0,39), continuous_update=False, prediction_filepath = filepath)

interactive(children=(Text(value='../Random Mess/Final/compare/felzenzsedge/', description='filepath'), IntSli…

<function __main__.compare(filepath, imagen0)>

In [21]:
# loop through all images and print the instance segmentation metrics
for i in range(40):
    TP, FP, FN= compare(filepath, i)
    print(TP,FP,FN)



4 0 0
4 0 0
10 1 2
1 3 10
4 3 3
3 1 0
3 5 18
21 1 5
16 2 4
2 2 8
18 2 1
4 0 0
4 1 0
4 2 0
4 0 0
4 1 0
0 0 28
31 0 0
17 7 11
0 0 26
6 4 13
7 7 16
5 3 5
2 18 103
2 13 188
16 11 74
12 0 0
25 5 13
15 1 5
7 8 28
10 2 2
6 4 22
20 4 9
11 4 0
20 3 4
16 7 12
24 4 6
5 1 0
6 1 2
9 8 60
